<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [37]:
#actualização de nomes de ficheiros e filtros

cadastro_nome="CadastroMensal_ONE_2024'Abril_desbl.xlsb"
servicos_nome="ONE_abr24.xlsb"
modelo_upfront_nome="Modelo upfront.xlsb"

#actualização do nome de folhas
servicos_folha="Base_Auto"
cadastro_folha="Cadastro de Ativos"

In [ ]:
#UPFRONT
#negócio -> marcações
  #marcações activações - modelo upfront 1,50€
  #marcações acrescentar activações - modelo upfront 1,50€
  #marcações retirar correcções marcações - modelo upfront + campanha - se for "a retirar" multiplica-se por -1

In [36]:
#carregar ficheiros
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()

Saving CadastroMensal_ONE_2024'Abril_desbl.xlsb to CadastroMensal_ONE_2024'Abril_desbl.xlsb


In [122]:
#função para retirar blanks e transformar em letras minúsculas
def formatar_palavra (palavra):
  return palavra.lower().strip()


0        mjpereira
1         fnaleixo
2        mjpereira
3         fnaleixo
4       gaferreira
           ...    
8900    masalcedas
8901       bflopes
8902     msfragoso
8903     fimacosta
8904      pfinacio
Name: user, Length: 8905, dtype: object

In [123]:
#ler ficheiro das activações
activacoes_df=pd.read_excel(servicos_nome, sheet_name=servicos_folha)

#formatar coluna "CAV" na listagem de activações
activacoes_df["CAV"]=activacoes_df.loc[activacoes_df["CAV"].isna()==False, "CAV"].apply(lambda i: str(i).split(".")[0])

#criar coluna "user 2" na listagem de activações
activacoes_df["user 2"]=activacoes_df["user"].apply(lambda i: formatar_palavra(i))
activacoes_df.head()

,Mês de Fecho,Order Item,CAV,SFID,Parceiro,Tipo,Negócio,Service Account,Customer Account (SA),CA&UA,...,GCO €,Valor final,Valor agregado,Valor com factor,Analise DAF,#,Grupo,Tipo.1,sub-tipo,user 2
0,45383,1-KR48OO5,80048024,L.G01.UP1.484,Heading,Ciclo - Alarmes,ALARME,S965668605,C525312301,C52531230110017778181,...,NaN,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,mjpereira
1,45383,1-KT44UBW,60138077,B.C01.UP2.0CH,Randstad,Ciclo - Alarmes,ALARME,S965769790,C656661301,C65666130110017881782,...,NaN,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,fnaleixo
2,45383,1-KR5XYMQ,80048024,L.G01.UP1.484,Heading,Ciclo - Alarmes,ALARME,S965671252,C688477401,C68847740110017788310,...,NaN,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,mjpereira
3,45383,1-KS2XK9F,60138077,B.C01.UP2.0CH,Randstad,Ciclo - Alarmes,ALARME,S965717921,C831052669,C83105266910017828793,...,NaN,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,fnaleixo
4,45383,1-KRU09AE,80161754,L.G01.UP1.625,Heading,Ciclo - Alarmes,ALARME,S965705815,C841003671,C84100367110017817872,...,NaN,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,gaferreira


In [124]:
#ler ficheiro cadastro
cadastro_df=pd.read_excel(cadastro_nome, sheet_name=cadastro_folha)
#criar coluna "User Aplicações 2" com letras minúsculas - criar nova coluna
cadastro_df["user aplicações 2"]=cadastro_df["User Aplicações"].apply(lambda i: formatar_palavra(i))

cadastro_df.head(2)

,Nome,Nº Colaborador,User Aplicações,Userde Login/consolas,CAV Inbound,CAV Outbound,CAV Alternativo,Data de Inicio atendimento,Data fim de actividade,Motivo de rescisão,...,Data Inicio de Campanha,Data Fim de Campanha,Campanha Final,Faltas,Férias,Elegível GCO GE,Rookie Elegível para GCO 100%,Parceiro,SFID´S,user aplicações 2
0,Patrícia Isabel Teixeira Moreira,80043208,U80043208,iqn,NaN,80043208,NaN,41472,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Randstad,X.V03.IN1.109,u80043208
1,Ana Sofia Rosinha Ferreira,80125104,anasofferreira,asr,NaN,80125104,NaN,43514,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heading,L.G01.UP1.322,anasofferreira


In [125]:
#ler ficheiro modelo upfront - matriz auto

matriz_auto=pd.read_excel(modelo_upfront_nome, sheet_name=0).iloc[:, [23, 24]]
matriz_auto.columns=["matriz", "auto"]

matriz_auto.head()


,matriz,auto
0,Campanha Final,Auto
1,lm_churn,Upsell
2,lm_churn_ac,Upsell
3,prospect_blitz_blm,Prospect
4,prospect_m_c,Prospect


In [126]:
#criar tabela "cadastro_auto" para criar coluna auto no ciclo de activações

#certificar que cada cav outbound tem uma única "campanha final"
print("o nº de campanhas finais por cav são: ")
print(cadastro_df.groupby("user aplicações 2") ["Campanha Final"].nunique().unique())

cadastro_auto=cadastro_df[["user aplicações 2", "Campanha Final"]].drop_duplicates("user aplicações 2", ignore_index=True)

cadastro_auto.head()

o nº de campanhas finais por cav são: 
[1 0]


,user aplicações 2,Campanha Final
0,u80043208,NaN
1,anasofferreira,NaN
2,u80047459,NaN
3,janabreu,NaN
4,apimoreira,NaN


In [127]:
#criar coluna provisória "campanha final" para determinar o auto de cada cav

activacoes_df["campanha final"]=activacoes_df.merge(cadastro_auto, how="left", left_on="user 2", right_on="user aplicações 2").iloc[:, -1]


activacoes_df.head()

,Mês de Fecho,Order Item,CAV,SFID,Parceiro,Tipo,Negócio,Service Account,Customer Account (SA),CA&UA,...,Valor final,Valor agregado,Valor com factor,Analise DAF,#,Grupo,Tipo.1,sub-tipo,user 2,campanha final
0,45383,1-KR48OO5,80048024,L.G01.UP1.484,Heading,Ciclo - Alarmes,ALARME,S965668605,C525312301,C52531230110017778181,...,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,mjpereira,up_alarmes_excl
1,45383,1-KT44UBW,60138077,B.C01.UP2.0CH,Randstad,Ciclo - Alarmes,ALARME,S965769790,C656661301,C65666130110017881782,...,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,fnaleixo,up_alarmes_excl
2,45383,1-KR5XYMQ,80048024,L.G01.UP1.484,Heading,Ciclo - Alarmes,ALARME,S965671252,C688477401,C68847740110017788310,...,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,mjpereira,up_alarmes_excl
3,45383,1-KS2XK9F,60138077,B.C01.UP2.0CH,Randstad,Ciclo - Alarmes,ALARME,S965717921,C831052669,C83105266910017828793,...,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,fnaleixo,up_alarmes_excl
4,45383,1-KRU09AE,80161754,L.G01.UP1.625,Heading,Ciclo - Alarmes,ALARME,S965705815,C841003671,C84100367110017817872,...,0.0,0.0,NaN,Comissionado Alarme,1,Alarme,Alarmes,Base,gaferreira,up_alarmes_excl


In [140]:
#determinar quantos users não têm campanha final

print(f"existem {activacoes_df.iloc[:, -1].isna().sum()} linhas sem campanha final determinada")
activacoes_df.loc[activacoes_df.iloc[:, -1].isna()==True, ["Tipo","user", "campanha final"]].drop_duplicates()


existem 8 users sem campanha final determinada


,Tipo,user,campanha final
8681,Ativações Negativas - Serviços,sedinis,NaN
8806,Qualidade VOC,U80043208,NaN
8807,Qualidade VOC,anasofferreira,NaN
8808,Qualidade Conf Expl,U80043208,NaN
8809,Qualidade Conf Expl,anasofferreira,NaN
8810,Qualidade IQ,U80043208,NaN
8811,Qualidade IQ,anasofferreira,NaN
8836,Qualidade IQ,mvgoncalves,up_pa


In [141]:
#determinar a coluna auto com base na coluna "campanha final"

for i, j in zip(matriz_auto.iloc[:, 0], matriz_auto.iloc[:, 1]):
  activacoes_df.loc[activacoes_df["campanha final"]==i, "auto"]=j

print(f"existem {activacoes_df['auto'].isna().sum()} linhas sem auto determinado")


activacoes_df.auto.head()

existem 8 linhas sem auto determinado


0    Alarmes
1    Alarmes
2    Alarmes
3    Alarmes
4    Alarmes
Name: auto, dtype: object

In [159]:
#criar coluna "sub-area de negócio no pressuposto de esta ainda não existir"
def sub_area_negocio(db, tipo, col_negocio, col_componente):
  if len([i for i in db.columns if i.startswith("Sub-")])==0:
    db2=db[db["Tipo"]==tipo]
    for i in db[col_negocio].unique():
      if i=="BLM":
        db2.loc[db2[col_negocio]==i, "Sub-Área de Negócio"]="NET MÓVEL"
      elif i=="MOBILE":
        db2.loc[db2[col_negocio]==i, "Sub-Área de Negócio"]="PÓS-PAGO"
      elif i=="NET":
        db2.loc[db2[col_negocio]==i, "Sub-Área de Negócio"]="NET"
      elif i=="VOZ FIXA":
        db2.loc[db2[col_negocio]==i, "Sub-Área de Negócio"]="VOIP"
      elif i=="TV":
        for j in db[col_componente].unique():
          if j=="Max":
            db2.loc[db2[col_componente]==j, "Sub-Área de Negócio"]="DTH"
          else:
            db2.loc[db2[col_componente]==j, "Sub-Área de Negócio"]="CABO"
  return db2

activacoes_df=sub_area_negocio(activacoes_df, "Ciclo - Serviços", "Negócio", "Componente")

activacoes_df.head()

<ipython-input-159-6dad1d26f38b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db2.loc[db2[col_componente]==j, "Sub-Área de Negócio"]="CABO"


,Mês de Fecho,Order Item,CAV,SFID,Parceiro,Tipo,Negócio,Service Account,Customer Account (SA),CA&UA,...,Valor com factor,Analise DAF,#,Grupo,Tipo.1,sub-tipo,user 2,campanha final,auto,Sub-Área de Negócio
11,45383,1-KT8GP9F,80160154,L.G01.UP1.603,Heading,Ciclo - Serviços,TV,S965775839,C142603002,C14260300210011527776,...,NaN,Comissionado,1,GA,Wired - TV,Base,ancocosta,prospect_pos,Prospect,CABO
12,45383,1-KQT7N5A,80113086,B.C01.UP1.206,Randstad,Ciclo - Serviços,TV,S965653029,C153861301,C1538613017916339077,...,NaN,Comissionado,1,GA,Wireless - TV,Base,jcamartins,prospect_teste_cob,Prospect,DTH
13,45383,1-KRUOFP1,80079190,L.G01.UP1.120,Heading,Ciclo - Serviços,TV,S965708068,C173627903,C1736279037900776644,...,NaN,Comissionado,1,GA,Wired - TV,Base,saucosta,prospect_pos,Prospect,CABO
14,45383,1-KSFAUGG,80165996,B.C01.UP1.582,Randstad,Ciclo - Serviços,TV,S220761201,C220761201,C2207612012038840458,...,NaN,Comissionado,1,GA,Wired - TV,Base,ccfrancisco,prospect_pre,Prospect,CABO
15,45383,1-KQBOQVE,80047659,L.G01.UP1.011,Heading,Ciclo - Serviços,TV,S965627485,C237169101,C2371691017916304102,...,NaN,Comissionado,1,GA,Wireless - TV,Base,u80047659,prospect_m_c,Prospect,DTH


In [150]:
#activacoes_df.columns
#activacoes_df[["Tipo", "Negócio", "Upfront €"]]

#analisar

#máscara coluna "tipo"
tipo_mask=activacoes_df["Tipo"].str.contains("serviço", case=False) & activacoes_df["Tipo"].str.contains("ciclo", case=False)

#máscara coluna negócio - excluir alarmes
negocio_mask1=activacoes_df["Negócio"].str.contains("alarme", case=False)

daf_mask1=activacoes_df["Analise DAF"].str.contains("dth para cabo", case=False)
daf_mask2=activacoes_df["Analise DAF"].str.contains("pré-pago", case=False)


#adicionar coluna "sub-área de negócio" (caso não exista)

activacoes_df.loc[(tipo_mask) & (~negocio_mask1) & (~daf_mask1) & (~daf_mask2),"Sub-Área de Negócio"]



KeyError: 'Sub-Área de Negócio'

In [155]:
activacoes_df.columns

Index(['Mês de Fecho', 'Order Item', 'CAV', 'SFID', 'Parceiro', 'Tipo',
       'Negócio', 'Service Account', 'Customer Account (SA)', 'CA&UA',
       'Alojamento Id', 'Componente', 'ID Componente', 'ID Pacote', 'Pacote',
       'REF DATA', 'BA', 'Nota SDD', 'Carga Horária', 'user', 'Data Ativação',
       'Data de Venda', 'Unidade de Alojamento', 'ID do Trabalho',
       'Tipo de Trabalho', 'Município', 'Célula', 'Segmento Unificado da SA',
       'SubSegmento Unificado da SA',
       'Data Início Período de Fidelização (SA)',
       'Data Fim Período de Fidelização (SA)',
       'Service Account - BI e NIF - BI', 'Service Account - BI e NIF - NIF',
       'MSISDN SA Mobile (a)', 'Campanha wise',
       'Campanha outbound (por CAV, Campanha Final)',
       'Função/SR_DATA_HORA_ENTRADA_PARTICIPACAO', 'Supervisor/QT_GA_LIQ2',
       'Valor Supervisor', 'Elegivel GCO CE/QT_CH_LIQ', 'TC', 'Valor TC',
       'Coordenador', 'Valor Coordenador', 'Analise / Pedido IB', 'GCO',
       'Notas DS'

In [28]:
activacoes_df.loc[(tipo_mask1) & (tipo_mask2), "Negócio"].unique()

array(['TV', 'NET', 'BLM', 'MOBILE', 'VOZ FIXA'], dtype=object)

In [27]:
[i for i in activacoes_df if i.find("Negócio")!=-1]

['Negócio']

PARTE ANTIGA

In [ ]:
#actualização de nomes de ficheiros e filtros

activacao_negativa="Ativações_Negativas_fev24_em_mar24_ONE.xlsb"

cadastro="Cadastro.xlsb" #copia-se o cadastro original para um ficheiro à parte antes de importar

capitalizacao="02.2024 - Capitalização_Eq. Interna Obv.xlsb"

filtro_cap="Activação"

In [ ]:
#carregar e definir ficheiro de activações negativas

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()


activacao_negativa_df=pd.read_excel(activacao_negativa, sheet_name=0)
cadastro_df=pd.read_excel(cadastro, sheet_name=0)
capitalizacao_df=pd.read_excel(capitalizacao, sheet_name="act")

Saving Ativações_Negativas_fev24_em_mar24_ONE.xlsb to Ativações_Negativas_fev24_em_mar24_ONE (4).xlsb


Saving Cadastro.xlsb to Cadastro (4).xlsb


In [ ]:
#função para criar a coluna cav e adicioná-la à bd de activações negativas

extensao_total="40168941189470163540"

def extractor (str1):

  dicionario_extensao={"entidade": 6, "delegacao": (6,12), "cav": 12}

  for i, j in dicionario_extensao.items():
    if i=="delegacao":
      start_index, end_index=j
      delegacao=str1[start_index:end_index]
    elif i=="cav":
      start_index=j
      cav=str1[start_index:]
    else:
      entidade=str1[:j]
  return cav


activacao_negativa_df["cav"]=activacao_negativa_df["CONCATENATE"].apply(extractor).astype("int")



activacao_negativa_df["cav"].head()


0    70163540
1    70163540
2    80077863
3    80130177
4    80123513
Name: cav, dtype: int64

In [ ]:
#eliminar duplicados no ficheiro do cadastro

cadastro_short=cadastro_df[["Cav outbound", "User Aplicações"]].drop_duplicates()


In [ ]:
#criar a coluna username na listagem de activações negativas


#converter cav outbound numa coluna "string"

#cadastro_short["Cav outbound"]=cadastro_short["Cav outbound"].astype("str")

activacao_negativa_df2= \
activacao_negativa_df.merge(cadastro_short[["Cav outbound", "User Aplicações"]], right_on="Cav outbound", left_on="cav", how="left")

activacao_negativa_df2.head()


print(f"existem {activacao_negativa_df2['User Aplicações'].isna().sum()} valores nulos na dataframe")

print(activacao_negativa_df2.shape)

activacao_negativa_df2.drop("Cav outbound", axis=1) #eliminar coluna "Cav outbound" que vem do cadastro

activacao_negativa_df2.loc[activacao_negativa_df2["cav"].isna()==True, ["cav", "Cav outbound"]]

existem 4 valores nulos na dataframe
(131, 110)


,cav,Cav outbound
32,70135131,NaN
33,70135131,NaN
72,80153411,NaN
114,80166426,NaN


In [ ]:
#filtrar capitalização por linhas de activação negativa

capitalizacao_df2=capitalizacao_df.loc[capitalizacao_df["Tipo.1"]==filtro_cap, ["Order Item", "valor unit pelos pontos Op", \
                    "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                    "valor unit pelos pontos Co"]] #Tipo.1 é o nome da coluna que tem o tipo de movimento

#criar colunas que vêm da capitalização: quantidade op, quantidade sup, quantidade tc, quantidade coord, quantidade coord2

activacao_negativa_df3=activacao_negativa_df2.merge(capitalizacao_df2, on="Order Item", how="left")

activacao_negativa_df3.shape



(131, 114)

In [ ]:
#multiplicar os valores das activações por -1 (porque são negativas)

activacao_negativa_df3=\
activacao_negativa_df3.loc[:, ["valor unit pelos pontos Op", "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                           "valor unit pelos pontos Co"]].apply(lambda j: j*-1)

In [ ]:
activacao_negativa_df3.to_excel("output.xlsx", index=False)

In [ ]:
#fazer download do ficheiro

files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>